In [9]:
import os
import sys
PATH="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations"
sys.path.append( os.path.join( PATH ) )

from allennlp.predictors.predictor import Predictor
import spacy

BERT_PATH=os.path.join(  PATH, "tests/test_files/models", "bert_model", "bert-base-srl-2019.06.17.tar.gz" )
SPACY_PATH=os.path.join( PATH, "tests/test_files/models", "spacy_model", "spacy-textcat" )

#print( f"loading AllenNLP predictor from {BERT_PATH}" )
#bert_model = Predictor.from_path( BERT_PATH, cuda_device=0 )

#print( f"loading spacy model from {SPACY_PATH}" )
#nlp=spacy.load( SPACY_PATH )

#http://docs.allennlp.org/v0.9.0/api/allennlp.predictors.html
from src.reporting_obligations import ReportingObligationsFinder

reporting_obligations_finder = ReportingObligationsFinder(  BERT_PATH, SPACY_PATH, gpu=0  )

loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/bert_model/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/spacy_model/spacy-textcat


In [10]:
#read in the json:
import json
import base64
import re
import sys
import os

from typing import Generator, List

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

TEMPLATE_PATH=os.path.join( PATH, "tests/test_files/templates/out.html.template" )


with open( os.path.join( PATH , "tests/test_files/typesystems/typesystem_user.xml" )  , 'rb') as f:
    typesystem = load_typesystem(f)

with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/double_nested_list_response.json')) as json_file:
    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/small_nested_tables_response.json')) as json_file:
#    response = json.load(json_file)
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/minus_lesser_of_response.json')) as json_file:
#    response = json.load(json_file)  

#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32011D1208-input.json')) as json_file:
#    response = json.load(json_file)  
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/32002R0063-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)    #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32014R1141-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)   #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32019L2121-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)   #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32019L2121-input.xmi_outputRO_gpu_2.json')) as json_file:
#    response = json.load(json_file)

#32019L2121-input.xmi_outputRO_gpu_2
    
#with open(  os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
#    response = json.load(json_file)
    
#OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/testing_bug3.html" )
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem )

RO_type=typesystem.get_type( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Reportingobligation" )
cas.get_view( "html2textView" ).add_annotation( RO_type(  begin=2292, end=11397, user='me' ) )
#what if two RO added... stay in sinc
cas.get_view("html2textView").select( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Reportingobligation" )


[de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Reportingobligation(xmiID=9543, id=None, user='me', datetime=None, begin=2292, end=11397, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Reportingobligation')]

In [11]:
from src.transform import ListTransformer

#from src.transform import get_other_lines, transform_lines, flatten_offsets

#from src.utils import SeekableIterator

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

transformer = ListTransformer( cas )
print( "add list view" )
transformer.add_list_view_user( OldSofaID='html2textView', NewSofaID = 'ListView' )

add list view


In [12]:


OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/testing_user.html" )

print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder.process_sentences( cas, ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( cas, TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

get reporting obligations
loading tokenizer en_core_web_sm
Writing output to /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/output_reporting_obligations/testing_user.html using /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/templates/out.html.template as html template
0.3407313823699951


In [ ]:
#END

In [6]:

RO_users=cas.get_view("html2textView").select( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Reportingobligation" )
#Check if they do not overlap.
RO_users

tags_users=[]
end=0
for RO_user in RO_users:
    if RO_user.begin>end or end==0: #check that the RO user annotations do not overlap
        tags_users+=list(cas.get_view( "html2textView" ).select_covered( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType", RO_user   ))
        end=RO_user.end
    else: 
        print( f"Reporting obligation annotated by user {RO_user.user} overlaps with another reporting obligation user annotation. \
        Please remove overlapping user annotations."   )


from src.utils import SeekableIterator
from src.transform import get_other_lines, flatten_offsets, postprocess_nested_lines, transform_lines

OldSofaID = "html2textView"
NewSofaID=  "ListView"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

seek_vbtt=SeekableIterator( iter( tags_users ) )

lines, offsets=get_other_lines( cas , OldSofaID, seek_vbtt, 'root', paragraph_type=paragraph_type )


In [8]:
lines[0]

['1.   Institutions shall report to the competent authorities, in accordance with the reporting requirements set out in Article 415(1) and the uniform reporting formats referred to in Article 415(3), the following items and their components in order to allow an assessment of the availability of stable funding:',
 '(a)',
 ['the following own funds, after deductions have been applied, where appropriate:',
  '(i)',
  'tier 1 capital instruments;',
  '(ii)',
  'tier 2 capital instruments;',
  '(iii)',
  'other preferred shares and capital instruments in excess of Tier 2 allowable amount having an effective maturity of one year or greater;'],
 '(b)',
 ['the following liabilities not included in point (a):',
  '(i)',
  'retail deposits that qualify for the treatment set out in Article 421(1);',
  '(ii)',
  'retail deposits that qualify for the treatment set out in Article 421(2);',
  '(iii)',
  'deposits that qualify for the treatment set out in Article 422 (3) and (4);',
  '(iv)',
  'of the

In [11]:
offsets

[[(2292, 2601),
  (2665, 2668),
  [(2710, 2789),
   (2937, 2940),
   (3018, 3045),
   (3275, 3279),
   (3357, 3384),
   (3614, 3619),
   (3697, 3833)],
  (4013, 4016),
  [(4058, 4110),
   (4258, 4261),
   (4339, 4412),
   (4642, 4646),
   (4724, 4797),
   (5027, 5032),
   (5110, 5185),
   (5415, 5419),
   (5497, 5747),
   (5977, 5980),
   (6058, 6152),
   (6382, 6386),
   (6464, 6558),
   (6788, 6793),
   (6871, 6981),
   (7211, 7217),
   (7295, 7341),
   (7571, 7575),
   [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)],
   (9146, 9149),
   (9227, 9393),
   (9623, 9627),
   [(9705, 9803),
    (10035, 10036),
    (10150, 10245),
    (10607, 10608),
    (10722, 10816)],
   (11176, 11181),
   (11259, 11281)]]]

In [3]:
from src.transform import ListTransformer

#from src.transform import get_other_lines, transform_lines, flatten_offsets

#from src.utils import SeekableIterator

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

transformer = ListTransformer( cas )
print( "add list view" )
transformer.add_list_view_user( OldSofaID='html2textView', NewSofaID = 'ListView' )

add list view
[[(2292, 2601), (2665, 2668), [(2710, 2789), (2937, 2940), (3018, 3045), (3275, 3279), (3357, 3384), (3614, 3619), (3697, 3833)], (4013, 4016), [(4058, 4110), (4258, 4261), (4339, 4412), (4642, 4646), (4724, 4797), (5027, 5032), (5110, 5185), (5415, 5419), (5497, 5747), (5977, 5980), (6058, 6152), (6382, 6386), (6464, 6558), (6788, 6793), (6871, 6981), (7211, 7217), (7295, 7341), (7571, 7575), [(7653, 7835), (8067, 8068), (8182, 8274), (8636, 8637), (8751, 8786)], (9146, 9149), (9227, 9393), (9623, 9627), [(9705, 9803), (10035, 10036), (10150, 10245), (10607, 10608), (10722, 10816)], (11176, 11181), (11259, 11281)]]]


In [ ]:
print( transformer.cas.get_view( "ListView" ).sofa_string )

In [10]:
RO_users=cas.get_view("html2textView").select( "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Reportingobligation" )
#Check if they do not overlap.
RO_users

[de_tudarmstadt_ukp_dkpro_core_api_segmentation_type_Reportingobligation(xmiID=9543, id=None, user='me', datetime=None, begin=2292, end=11397, type='de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Reportingobligation')]

In [41]:
tags_users=[]
end=0
for RO_user in RO_users:
    if RO_user.begin>end or end==0: #check that the RO user annotations do not overlap
        tags_users+=list(cas.get_view( "html2textView" ).select_covered( "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType", RO_user   ))
        end=RO_user.end
    else: 
        print( f"Reporting obligation annotated by user {RO_user.user} overlaps with another reporting obligation user annotation. \
        Please remove overlapping user annotations."   )

In [37]:
from src.utils import SeekableIterator
from src.transform import get_other_lines, flatten_offsets, postprocess_nested_lines, transform_lines

OldSofaID = "html2textView"
NewSofaID=  "ListView"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

seek_vbtt=SeekableIterator( iter( tags_users ) )

lines, offsets=get_other_lines( cas , OldSofaID, seek_vbtt, 'root', paragraph_type=paragraph_type )


In [38]:
flatten_offsets( offsets )

lines, offsets =postprocess_nested_lines( lines, offsets  )

assert len( lines ) == len( offsets )

transformed_lines, transformed_lines_offsets=transform_lines( lines, offsets )

assert len( transformed_lines ) == len( transformed_lines_offsets )

lines_offsets=[]
for line, offset in zip( transformed_lines, transformed_lines_offsets  ):
    lines_offsets.append(line + "|" + str( offset ))

#add the transformed lines to the cas

cas.create_view(NewSofaID)

cas.get_view( NewSofaID).sofa_string = "\n".join( lines_offsets )

In [40]:
print(cas.get_view( "ListView" ).sofa_string )

1.   Institutions shall report to the competent authorities, in accordance with the reporting requirements set out in Article 415(1) and the uniform reporting formats referred to in Article 415(3), the following items and their components in order to allow an assessment of the availability of stable funding ❮the following own funds, after deductions have been applied, where appropriate ❬tier 1 capital instruments ‖ and/or tier 2 capital instruments ‖ and/or other preferred shares and capital instruments in excess of Tier 2 allowable amount having an effective maturity of one year or greater❭ ‖ the following liabilities not included in point (a) ❬retail deposits that qualify for the treatment set out in Article 421(1) ‖ and/or retail deposits that qualify for the treatment set out in Article 421(2) ‖ and/or deposits that qualify for the treatment set out in Article 422 (3) and (4) ‖ and/or of the deposits referred to in point (iii), those that are subject to a deposit guarantee scheme i